In [1]:
import pdfquery


In [7]:
pdf = pdfquery.PDFQuery("data/112817.pdf")
pdf.load()

In [8]:
# see what happens when we dump it
filename = "tmp.xml"
pdf.tree.write(filename, pretty_print=True)

In [13]:
[t.text for t in pdf.tree.findall(".//LTTextLineHorizontal")]

['Francis D. Riley ',
 'Chief Of Police ',
 '',
 '1033 Massachusetts Avenue ',
 'Sixth Floor ',
 'Cambridge, Massachusetts 02138 ',
 'Phone: 617-495-1215 ',
 'Fax: 617-495-7782 ',
 'President and Fellows of Harvard College ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Officer dispatched to a report of an unwanted guest sitting in on a class. Officer arrived, located individual and ',
 'conducted a field interview. The individual was run for wants/warrants with negative results. The individual was then ',
 'advised that they need to get permission to sit in on future classes. The individual was then sent on their way. ',
 '11/28/17 ',
 '3:14 PM ',
 '',
 '11/16/17 ',
 '10:08 AM ',
 '',
 '',
 '',
 '',
 '',
 '',
 '11/28/17 ',
 '8:26 AM ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '11/28/17 ',
 '9:08 AM ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '11/28/17 ',
 '10:17 AM ',
 '',
 '11/27/17 - 5:45 PM ',
 '11/28/17 - 9:00 AM ',
 '',
 '',
 '',
 '',
 '',
 '11/28/17 ',
 '12:07 PM ',
 '',


In [14]:
[t.text for t in pdf.tree.findall(".//LTTextBoxHorizontal")]

['',
 'Harvard University Police Department ',
 '',
 u'Copyright \xa9 ',
 '2017 ',
 'PUBLIC POLICE LOG ',
 '11/28/2017 ',
 'Date & Time ',
 'Date & Time ',
 'Officer dispatched to take a report of a stolen U.S. currency valued at $620.00. ',
 'Officer dispatched to take a report of a stolen blue Canada Goose jacket valued at $1,000.00. ',
 'Officer dispatched to take a report of a stolen green Subaru Forrester valued at $12,000.00. ',
 'Officer dispatched to take a report of threats. ',
 '',
 '',
 'Officer dispatched to take a report of a stolen package containing an HP laptop valued at $821.00. ',
 '',
 '1 WESTERN AVE ',
 'Reported ',
 'Incident Type ',
 'Occurred ',
 'Location ',
 'Disposition ',
 '',
 'THEFT REPORT ',
 '11/28/17 ',
 '7:30 AM - 8:00 AM ',
 'MALKIN ATHLETIC CENTER ',
 '39 HOLYOKE ST ',
 'CAMBRIDGE ',
 'CLOSED ',
 ' ',
 '',
 'THEFT REPORT ',
 '11/21/17 ',
 '2:00 PM - 6:00 PM ',
 'NEW DENTAL RESEARCH BUILDING ',
 '190 LONGWOOD AVE ',
 'BOSTON ',
 'CLOSED ',
 ' ',
 '',
 

In [15]:
# so each individual report, as well as headers, is filed inside 
# its own <LTCurve>. The text fields are inside <LTTextLineHorizontal>s and <LTTextBoxHorizontal>s.

In [17]:
reports_plus_heads = pdf.tree.findall(".//LTCurve")

In [18]:
report2 = reports_plus_heads[1]

In [19]:
report2

<Element LTCurve at 0x1083e5788>

In [28]:
cleaned_texts

['11/28/17',
 '8:26 AM',
 'THEFT REPORT',
 '11/28/17',
 '7:30 AM - 8:00 AM',
 'MALKIN ATHLETIC CENTER',
 '39 HOLYOKE ST',
 'CAMBRIDGE',
 'CLOSED']

In [32]:
def get_text_from_curve(ltcurve):
    textual_elements = (ltcurve.findall(".//LTTextLineHorizontal") + 
                        ltcurve.findall(".//LTTextBoxHorizontal"))
    texts = [t.text.strip() for t in textual_elements]
    
    # remove empty lines
    cleaned_texts = [t for t in texts if t != '']
    
    return cleaned_texts

In [35]:
HEADER_ROW_TEXT = ['Reported', 'Incident Type', 'Occurred', 'Location', 'Disposition']

In [36]:
incidents = [get_text_from_curve(lt) for lt in reports_plus_heads]

# remove headers
incidents_without_headers = [i for i in incidents if i != HEADER_ROW_TEXT]

In [37]:
incidents_without_headers

[['11/28/17',
  '8:26 AM',
  'THEFT REPORT',
  '11/28/17',
  '7:30 AM - 8:00 AM',
  'MALKIN ATHLETIC CENTER',
  '39 HOLYOKE ST',
  'CAMBRIDGE',
  'CLOSED'],
 ['11/28/17',
  '9:08 AM',
  'THEFT REPORT',
  '11/21/17',
  '2:00 PM - 6:00 PM',
  'NEW DENTAL RESEARCH BUILDING',
  '190 LONGWOOD AVE',
  'BOSTON',
  'CLOSED'],
 ['11/28/17',
  '10:17 AM',
  '11/27/17 - 5:45 PM',
  '11/28/17 - 9:00 AM',
  'MOTOR VEHICLE THEFT',
  'FACULTY ROW CAR PORT',
  '46 LINNAEAN ST',
  'CAMBRIDGE',
  'CLOSED'],
 ['11/28/17',
  '12:07 PM',
  '11/27/17 - 4:53 PM',
  '11/28/17 - 11:02 AM',
  'THREAT(S)',
  'DUDLEY HOUSE - LEHMAN HALL',
  '8 HARVARD YARD',
  'CAMBRIDGE',
  'OPEN'],
 ['11/28/17',
  '1:48 PM',
  'FIELD INTERVIEW',
  '11/28/17',
  '1:48 PM - 2:31 PM',
  'ALDRICH HALL',
  '35 HARVARD WAY',
  'ALLSTON',
  'CLOSED'],
 ['THEFT REPORT', 'CLOSED', 'ALLSTON'],
 ['11/28/17',
  '4:20 PM',
  '11/28/17',
  '4:20 PM',
  'EMERSON HALL',
  '26 HARVARD YARD',
  'SUSPICIOUS ACTIVITY',
  'CAMBRIDGE',
  'CLOSED'],
